# Problem 2

## (a) 
### Hidden layer:

1D Input → use shallow network (1–2 hidden layers)

2D or more dimension input → may need deeper network

### Node number:

Smooth, simple function→ small width (16–32)

Oscillatory / curved function→ moderate width (32–64)

Very complex function → larger width or extra layer

For this question, I choose 1 hidden layer and 32 nodes

## (b)
### Hidden layers: 

use ReLU as a good first choice (remember, it's just a good default, not a mandatory one).

### Output layer:

Regression (predict a real number, e.g., stock price differences): use Linear output.

Regression (predict a non-negative number, e.g., heights): use ReLU output. A special case of this is - predicting values between 0 and 1, say Probabilities - use Sigmoid output.

Binary classification (yes/no): use Sigmoid output (one node).

Multi-class classification (one of many classes): use Softmax (not covered yet in class) - don’t use a single sigmoid for this.

### My answer:
Hidden layer: ReLU

output layer: Linear

## (c)



![Loss Function](loss_function.png)

based on this rule, I choose MSE

## (d)

## Import the data

In [62]:
import pandas as pd

In [63]:
df = pd.read_csv('hw7_data.csv')

## Split data into two columns 

In [64]:
x = df.iloc[:, 0].to_numpy()

In [65]:
y = df.iloc[:, 1].to_numpy()

## Nomralize the data

In [66]:
x_normalized = (x - x.mean()) / x.std()
y_normalized = (y - y.mean()) / y.std()

In [67]:
x_normalized.shape

(99,)

## Convert data into tensor

In [68]:
import torch

In [69]:
x_t = torch.tensor(x_normalized, dtype=torch.float32).view(-1, 1)
y_t = torch.tensor(y_normalized, dtype=torch.float32).view(-1, 1)

# Convert the normalized data into a 2-dimensional float tensor that the neural network can read and process.

## Dataset & Dataloader

In [70]:
from torch.utils.data import TensorDataset, DataLoader

In [71]:
dataset = TensorDataset(x_t, y_t)
# Combine input tensor (x_t) and target tensor (y_t) into a dataset
# Each sample becomes a pair: (x, y)


loader = DataLoader(dataset, batch_size=32, shuffle=True)
# shuffle the order of pairs of data in the dataset and then splist the dataset into mini-batches
# 99 data & 32 batch --> batch1 32 pairs of data, batch2 32 pairs of data
# batch3 32 pairs of data, batch4 3 pairs of data


# A small dataset generally means fewer than 10,000 samples, 
# while fewer than 1,000 samples is considered very small. 
# dataset with 99 samples is considered very small or tiny.
# For small datasets, it is recommended to use smaller batch sizes such as 8, 16, or 32.
# In general, smaller datasets prefer smaller batches, and larger datasets can handle larger batch sizes.

## Model

In [72]:
import torch.nn as nn

In [73]:
model = nn.Sequential(     # generate the NNs
    nn.Linear(1, 32),      # the first hidden layer with 32 neurons (you choose the num of neurons before)
    nn.ReLU(),             # activation function (activation functions introduce non-linearity into the model)
    nn.Linear(32, 1)       # the output layer 
)

## Loss & Optimizer

In [83]:
loss_fn = nn.MSELoss() 
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# There are many optimizers. You may want to learn how to choose the loss_f and optimizers

## Training Loop

For small datasets like yours (99 samples)

Try 200–1000 epochs

Small datasets often need more epochs because each epoch sees very little data

In [84]:
epochs = 500
for epoch in range(epochs):
    for x, y in loader:
        # Forward pass (model makes prediction)
        forw = model(x)

        # compute loss (measure prediction error)
        loss = loss_fn(forw, y)

        # Backward pass (compute gradients to know How much the loss will change if we change this weight)
        optimizer.zero_grad()
        loss.backward()

        # Update weights (Adjust weights to reduce loss actomatically)
        optimizer.step()

    if epoch % 50 == 0:
        print (f"Epoch {epoch}, Loss: {loss.item()}")

        # We print training information to monitor learning progress, detect problems,
        # and decide when to stop training.


Epoch 0, Loss: 0.4851933717727661
Epoch 50, Loss: 0.7577405571937561
Epoch 100, Loss: 0.8178079128265381
Epoch 150, Loss: 0.5439125895500183
Epoch 200, Loss: 0.8879322409629822
Epoch 250, Loss: 0.7554197311401367
Epoch 300, Loss: 0.8804777264595032
Epoch 350, Loss: 2.309202194213867
Epoch 400, Loss: 1.8837543725967407
Epoch 450, Loss: 1.344854712486267
